## Installing Unsloth library

In [ ]:
# Installing the Unsloth library
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Installing the other libraries required by unsloth to run
!pip install --no-deps xformers trl peft accelerate bitsandbytes triton

## Importing necessary libraries

In [ ]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

## Loading the model for inferencing

In [ ]:
max_seq_length = 2048 # Maximum number of token allowed in a sequence

# Loading the model for running inference from ressurectAI available at https://huggingface.co/ressurectAI/base
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="ressurectAI/base",
    max_seq_length=max_seq_length,
    dtype=None,
)
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "Person", "assistant": "Gandhi"},
    chat_template="chatml",
)
FastLanguageModel.for_inference(model)

In [7]:
# Function to generate response
def send_message(input):
  messages = [
    {"from": "Person", "value": "Gandhi ji, {}".format(input)},
  ]
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=False,
      return_tensors="pt",
  ).to("cuda")
  result = model.generate(input_ids=inputs,pad_token_id = tokenizer.pad_token_id , max_new_tokens=1024,use_cache=True)
  response = str(tokenizer.batch_decode(result)[0].split('assistant\n')[1].replace('<|im_end|>','').strip())
  return response

## Interact with the model

In [ ]:
while(True):
  user_input = input('User: ')
  print('Gandhiji: ',send_message(user_input))
  if input == 'exit':
    break